# Create a map of all protected_areas with a given protect_class and show contained ways (access set green, no access set red)

## Read data from overpass (or cached .json file)

In [ ]:
import requests, os, re
from datetime import datetime

import json
import osm2geojson
import folium

from utils.mapfolium import add_map_legend

In [ ]:
read_overpass = True
include_ways = True

dataFile = "data/overpass.json"
outHtml = "html-out/Schongebiete-Alpenrand-BY.html"

if not os.path.isdir('data'):
    os.mkdir('data')

if read_overpass:
    # https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example

    overpass_url = "http://overpass-api.de/api/interpreter"
    bbox = "47.378,11.078,47.768,13.111"
    pclass = "14"

    # way["highway"][!"access:conditional"][!"access"](area); -> get all contained ways but color them differently below
    # bbox: http://norbertrenner.de/osm/bbox.html
    overpass_query = f"""
    [out:json][bbox:{bbox}];
    (
    way["boundary"="protected_area"]["protect_class"="{pclass}"];
    relation["boundary"="protected_area"]["protect_class"="{pclass}"];
    );
    """
    if include_ways:
        dataFile = "overpass_ways.json"
        outHtml = "html-out/Schongebiete-Alpenrand-BY-Wege.html"
        overpass_query += f"""
    map_to_area;(
    way["highway"](area);
    way["boundary"="protected_area"]["protect_class"="{pclass}"];
    relation["boundary"="protected_area"]["protect_class"="{pclass}"];
    );
    """
    overpass_query += f"""
    out body;
    >;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    with open(f"data/{dataFile}", "w") as text_file:
        text_file.write(json.dumps(data))
else:
    with open(f"data/{dataFile}", "r") as text_file:
        data = json.loads(text_file.read())

## Postprocess the geojson
- build a feature.properties.tagsStr which can be picked up from folium.GeoJsonPopup

In [ ]:
# https://pypi.org/project/osm2geojson/
geojson = osm2geojson.json2geojson(data)

for feat in geojson['features']:
    id = feat['properties']['id']
    type = feat['properties']['type'] # 
    tagsStr = [f'<td><b>OSM ID:</b></td><td><a target="osm" href="https://www.openstreetmap.org/{type}/{id}"><b>{id}</b></a> / ' +
        f'<a target="josm" href="http://localhost:8111/load_object?new_layer=false&objects=w{id}"><b>JOSM</b></a></td>']
    tags = feat['properties']['tags']
    for tag in tags.keys():
        tagsStr.append(f"<tr><td>{tag}:</td><td>{tags[tag]}</td></tr>")
    feat['properties']['tagsStr'] = '<table border=all>'+''.join(tagsStr)+'</table>'

# with open("geo.json", "w") as text_file:
#     text_file.write(json.dumps(geojson))

## sort the geojson 
- so that all Polygons come first
- so that folion creates higher z-indexes for the LineString
- so that they are clickable

In [ ]:
def get_ftype(json): 
    try:
        return ord(json['geometry']['type'][0])
    except KeyError:
        return 0
list.sort(geojson['features'], key=get_ftype, reverse=True)

## Create the map

In [ ]:
def style_function(feature):
    if feature ['geometry']['type'] == 'Polygon':
        if ( ( 'access' in feature['properties']['tags'].keys() and re.match(r"^no.*", feature['properties']['tags']['access']) ) or
            ('access:conditional' in feature['properties']['tags'].keys() and re.match(r"^no.*", feature['properties']['tags']['access:conditional']) ) ):
            return {
                "fillOpacity": 0.5,
                "fillColor": "orange"
            }
        else:
            return {
                "fillOpacity": 0.5,
                "fillColor": "yellow"
            }
    elif feature['geometry']['type'] == "LineString":
        if 'access:conditional' in feature['properties']['tags'].keys() or 'access' in feature['properties']['tags'].keys():
            return {
                "color": "green"
            }
        else:
            return {
                "color": "red"
            }
    else:
        return {}

m = folium.Map(
    location=[47.68, 11.93],
    zoom_start=12
)

folium.GeoJson(
    geojson, 
    name="geojson", 
    style_function = style_function,
    popup = folium.GeoJsonPopup(
        fields=['tagsStr'],
        aliases=['']
    )    
).add_to(m)

colors = ['orange','yellow']
categories = ["Betretungs-Verbot", "Bitte um Nichtbetretung"]

if include_ways:
    colors += ['red','green']
    categories += ["Weg noch zu sperren", "Weg gesperrt"]

date_time = datetime.now().strftime("%d.%m.%Y, %H:%M:%S")
m = add_map_legend(m, f"<b>OSM Wald- und Wild-Schongebiete<br/>Bayerischer Alpenrand</b><br/><i>Stand: {date_time}</i><br/><br/>Kategorie", colors, categories)

# Add map title
# m.get_root().html.add_child(folium.Element(f"<h1>Wald- und Wild-Schutzgebiete</h1>"))

m.save(outHtml)